In [1]:
import torch
import torch.nn as nn
import torchvision.models as models
import torch.nn.functional as F

from pooling import GeM, SPoC, MAC

In [2]:
class Ensemble(nn.Module):
    def __init__(self):
        super().__init__()
        self.backbone = models.resnet50(pretrained=True)
        del self.backbone.avgpool, self.backvone.fc
        
        poolings = [GeM(), SPoC(), MAC()]
        
        for idx, module in enumerate(poolings):
            setattr(self, f'fc-{idx}', nn.Linear(2048,512))
            setattr(self, f'pool-{idx}', module)
        self.last_fc = nn.Linear(1536, 20)
        
    def forward(self, x):
        for _, module in self.backbone.named_children():
            x = module(x)
            
        xs =[]    
        for idx in range(3):
            fc = getattr(self, f'fc-{idx}')
            pool =getattr(self, f'pool-{idx}')
            
            x = pool(x)
            x = self.flatten(x)
            x = fc(x)
            x = F.normalize(x)
            xs.append(x)
            
        x = torch.cat(xs, 1)
        x = F.normalize(x)
        x = self.last_fc(x)
        return x